# Análisis e Implementación en SparkQL

* Entrega hasta el 24 de mayo en https://forms.gle/aRiWtjayausRFCTd8

* Los dos conjuntos de datos entregados, CSV separados por coma, Evaluación del Pitch (2021.04.13 ISoftware).csv y Asistencia del Pitch (2021.04.13 ISoftware).csv proceden de dos encuestas realizadas con Google Forms.

* Los archivos proceden de las evaluaciones de los pitch (exposiciones cortas de negocio) que realizan los estudiantes de la asignatura de Ingeniería de Software. Los estudiantes están organizados en equipos. Cada equipo realiza una presentación de máximo 15 minutos, una vez inicia la presentación todos los estudiantes deben registrar su asistencia lo cual queda registrado en 'Asistencia del Pitch (2021.04.13 ISoftware).csv' incluidos los miembros del equipo.

* El archivo `Asistencia del Pitch (2021.04.13 ISoftware).csv` contiene las columnas: `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto,`"Equipo al que perteneces:"` equipo de trabajo al que pertenece el estudiante también es texto y `"Equipo que va a exponer:"` equipo que el estudiante va a tender a su presentación.

* El archivo `Evaluación del Pitch (2021.04.13 ISoftware).csv` contiene las columnas `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto, `"Equipo que vas a evaluar:"` equipo que ha expuesto y que va a ser evaluado por cada estudiante que no sea integrante; a continuación, se tienen las siguientes columnas que corresponden a la evaluación de los respectivos ítems:

    1. "Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?",
    2. "Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?",
    3. "Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?",
    4. "Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?",
    5. "Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?",
    6. "Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.",
    7. "Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.",
    8. "Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.",
    9. "Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?",
    10. "Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ",
    11. "Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?",
    12. "Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.",
    13. "Exposición: ¿Qué tan coordinados estaban los expositores?",
    14. "Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?",
    15. "Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.",
    * "Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)",
    * "Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación."


* Cada ítem se evalúa con la siguiente escala: 0. Ausente; 1. Deficiente; 2. Regular; 3. Aceptable; 4. Bueno; 5. Excelente

* Carge los datos, cada archivo en una tabla SparkSQL y responda cada una de las consultas dadas en cada celda. Tenga en cuenta que algunas consultas pueden tener como resultado el vacío.

# Integrantes del equipo
1. 
2.
3.
4.
5.

In [2]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [3]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

!hdfs dfs -rm -f -r /tmp/data
!hdfs dfs -mkdir /tmp/data 

##!mv 'Asistencia del Pitch (2021.04.13 ISoftware).csv' 'asistencia.csv'
##!mv 'Evaluacion del Pitch (2021.04.13 ISoftware).csv' 'evaluacion.csv'

!hdfs dfs -copyFromLocal ./asistencia.csv  /tmp/data/asistencia.csv
!hdfs dfs -copyFromLocal ./evaluacion.csv  /tmp/data/evaluacion.csv

Deleted /tmp/data


In [34]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...

df = SesionSpark.read.load("/tmp/data/asistencia.csv",
                     format="csv",
                     sep=",",
                     inferSchema="true",
                     header="true")

df_asistencia = df.withColumnRenamed("Marca temporal","fecha")\
    .withColumnRenamed("Nombre de usuario","nombre_usuario")\
    .withColumnRenamed("Equipo al que perteneces:","equipo_pertenece")\
    .withColumnRenamed("Equipo que va a exponer:","equipo_expone")

df_asistencia.createOrReplaceTempView('asistencia')

df2 = SesionSpark.read.load("/tmp/data/evaluacion.csv",
                     format="csv",
                     sep=",",
                     inferSchema="true",
                     header="true")

df_evaluacion = df2.withColumnRenamed("Marca temporal","Fecha")\
    .withColumnRenamed("Nombre de usuario","nombre_usuario")\
    .withColumnRenamed("Equipo que vas a evaluar:","Equipo_evaluar")\
    .withColumnRenamed("Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?","introduccion")\
    .withColumnRenamed("Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?","equipo")\
    .withColumnRenamed("Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?","problema")\
    .withColumnRenamed("Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?","ventajas")\
    .withColumnRenamed("Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?","solucion")\
    .withColumnRenamed("Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.","producto")\
    .withColumnRenamed("Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.","traccion")\
    .withColumnRenamed("Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.","mercado")\
    .withColumnRenamed("Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?","team_in")\
    .withColumnRenamed("Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ","modelo_negocio")\
    .withColumnRenamed("Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?","inversion")\
    .withColumnRenamed("Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.","contacto")\
    .withColumnRenamed("Exposición: ¿Qué tan coordinados estaban los expositores?","exp_coordinacion")\
    .withColumnRenamed("Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?","exp_claridad")\
    .withColumnRenamed("Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.","exp_diapositivas")\
    .withColumnRenamed("Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)","pregunta_invertiria")\
    .withColumnRenamed("Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación.","observaciones")

df_evaluacion.createOrReplaceTempView('evaluacion')

In [35]:
ContextoSql.sql("SHOW TABLES").show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|        |asistencia|       true|
|        |evaluacion|       true|
+--------+----------+-----------+



In [42]:
df_evaluacion.printSchema()

root
 |-- Fecha: string (nullable = true)
 |-- nombre_usuario: string (nullable = true)
 |-- Equipo_evaluar: string (nullable = true)
 |-- introduccion: integer (nullable = true)
 |-- equipo: integer (nullable = true)
 |-- problema: integer (nullable = true)
 |-- ventajas: integer (nullable = true)
 |-- solucion: integer (nullable = true)
 |-- producto: integer (nullable = true)
 |-- traccion: integer (nullable = true)
 |-- mercado: integer (nullable = true)
 |-- team_in: integer (nullable = true)
 |-- modelo_negocio: integer (nullable = true)
 |-- inversion: integer (nullable = true)
 |-- contacto: integer (nullable = true)
 |-- exp_coordinacion: integer (nullable = true)
 |-- exp_claridad: integer (nullable = true)
 |-- exp_diapositivas: integer (nullable = true)
 |-- pregunta_invertiria: string (nullable = true)
 |-- observaciones: string (nullable = true)



In [43]:
df_asistencia.printSchema()

root
 |-- fecha: string (nullable = true)
 |-- nombre_usuario: string (nullable = true)
 |-- equipo_pertenece: string (nullable = true)
 |-- equipo_expone: string (nullable = true)



In [47]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

ContextoSql.sql("""

SELECT nombre_usuario FROM asistencia

GROUP BY nombre_usuario

ORDER BY nombre_usuario ASC

""").show()

+--------------------+
|      nombre_usuario|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
|cquinchiar@unal.e...|
| dadazam@unal.edu.co|
|daestradam@unal.e...|
|davgarciava@unal....|
|dballesteroso@una...|
|  dbrito@unal.edu.co|
|dcadavid@unal.edu.co|
|dcardonaal@unal.e...|
|dchavarriar@unal....|
|dgarciabl@unal.ed...|
+--------------------+
only showing top 20 rows



In [65]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

SesionSpark.sql("""
SELECT nombre_usuario, 
        COUNT(nombre_usuario) AS asistencia_usuario, 
        MAX(fecha) AS ultima_asistencia, 
        MIN(fecha) AS primer_asistencia 
FROM 
        asistencia 

GROUP BY 
        nombre_usuario

""").show()


+--------------------+------------------+--------------------+--------------------+
|      nombre_usuario|asistencia_usuario|   ultima_asistencia|   primer_asistencia|
+--------------------+------------------+--------------------+--------------------+
|samolinap@unal.ed...|                10|2021/04/13 9:37:2...|2021/04/08 8:11:4...|
| jruedat@unal.edu.co|                 8|2021/04/13 9:41:2...|2021/04/08 8:12:5...|
|nvalenciat@unal.e...|                 8|2021/04/13 9:39:3...|2021/04/08 8:10:1...|
|serendona@unal.ed...|                 8|2021/04/13 9:40:2...|2021/04/08 8:10:4...|
|jgutierrezlo@unal...|                 8|2021/04/13 9:39:0...|2021/04/08 8:12:0...|
|dgerenal@unal.edu.co|                 8|2021/04/13 9:41:2...|2021/04/08 8:10:0...|
|cjfunezg@unal.edu.co|                 8|2021/04/13 9:41:4...|2021/04/08 8:12:1...|
|emflorezb@unal.ed...|                 8|2021/04/13 9:45:4...|2021/04/08 8:14:0...|
|dchavarriar@unal....|                 8|2021/04/13 9:43:5...|2021/04/08 8:1

In [ ]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...


In [ ]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...



In [ ]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...


In [ ]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...


In [ ]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...


In [ ]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...


In [ ]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...


In [ ]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...


In [ ]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...
